In [1]:
import pandas as pd

C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
experience_df = pd.read_csv("./datasets/formats/ias-experience.csv")
education_df = pd.read_csv("./datasets/formats/ias-education.csv")

In [3]:
subjects_df = pd.read_csv("./processed/IAS subjects.csv")

In [4]:
subjects_df

,Subject,Count,Category
0,Economics,1815,Social Science and Liberal Arts
1,Law,956,"Governance, Law and Management"
2,History,930,Social Science and Liberal Arts
3,Political Sc.,903,Social Science and Liberal Arts
4,English,884,Languages and Fine Arts
...,...,...,...
490,Home Science,1,Others
491,Integrated Electronics,1,"Engineering, Technology and Mathematics"
492,Labour Legisln,1,"Governance, Law and Management"
493,Petroleum Eco.,1,Social Science and Liberal Arts


In [5]:
education_df.columns

Index(['ID', 'Name', 'Service', 'Cadre', 'Reference_Value', 'Qualification',
       'Subject', 'Division', 'Source'],
      dtype='object')

In [6]:
subjects_items = education_df["Subject"].copy()
for subject, category in zip(subjects_df["Subject"], subjects_df["Category"]):
    subjects_items.replace({subject: category}, inplace=True)

In [7]:
education_df["Category"] = subjects_items

In [8]:
education_df = education_df.loc[education_df["Category"] != "N.A."]

In [9]:
education_df.dropna(inplace=True)

In [10]:
names = education_df["Name"].unique()

In [13]:
from tqdm import tqdm

In [34]:
subject_lists = []
for name in tqdm(names):
    temp = education_df.loc[education_df["Name"] == name]
    subject_lists.append(temp["Category"].unique().tolist())

100%|██████████| 7105/7105 [00:08<00:00, 870.44it/s]


In [33]:
from itertools import product

In [86]:
subject_map = []
map_df = pd.DataFrame()
for i, name in tqdm(enumerate(names)):
    temp = experience_df.loc[experience_df["Name"] == name]
    temp_subject_list = [" | ".join(subject_lists[i])] * len(temp.index)
    temp["Subject_list"] = temp_subject_list
    map_df = map_df.append(temp, ignore_index=True)

0it [00:00, ?it/s]C:\Users\vedan\AppData\Local\Temp/ipykernel_10772/1319141189.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp["Subject_list"] = temp_subject_list
7105it [00:55, 128.81it/s]


In [87]:
final_map = map_df.groupby(["Category_of_Experience", "Subject_list"]).size().dropna().reset_index()

In [88]:
final_map.to_clipboard(index=False)